## xlsx joiner

Scipt to read xlsx files and join them as a simple csv

In [8]:
import datetime
import os
import re

import pandas

### File read and filter


In [9]:
def get_xlsx_files(directory):
    files_extension = "xlsx"
    current_path = os.getcwd()
    xlsx_files = list()
    year_dirs = os.listdir(f"{current_path}/{directory_name}")
    year_dirs.sort()
    for year_dir in year_dirs:
        file_names = os.listdir(f"{current_path}/{directory_name}/{year_dir}")
        file_names.sort()
        for file_name in file_names:
            if re.match(".*\.xlsx$", file_name):
                xlsx_files.append(
                    f"{current_path}/{directory_name}/{year_dir}/{file_name}"
                )
    return xlsx_files


In [10]:
def get_volume_df(filepath):
    volume = pandas.read_excel(filepath, engine='openpyxl', sheet_name=0)
    # Remove first usless header
    df = volume.drop([0])
    df.columns = range(df.shape[1])

    def get_street_id(x):
        match =  re.search('^(.*) \|', x)
        if match:
            return match.group(1)

    # Get header rows and fill null values
    headers = df[0:3].copy()
    headers.iloc[0] = headers.iloc[0].apply(lambda x: None if x == "Total" else x)
    headers = headers.apply(lambda x: x.ffill(), axis=1)
    headers.iloc[1] = headers.iloc[1].apply(get_street_id)

    cameras = df.iloc[3].fillna("Total")

    # Concatenate all headers in one row
    headers = headers.append(cameras)
    column_names = headers.loc[1] + "_" + headers.loc[2] + " " +headers.loc[4]
    column_names[0] = "date"
    column_names[1] = "time"
    values = df[[isinstance(x, datetime.datetime) for x in df[0]]].copy()
    values.columns = column_names
    values.reset_index(inplace=True, drop=True)
    values['date'] = [d.date() for d in values['date']]
    return values

In [11]:
def get_speed_df(filepath):
    speed = pandas.read_excel(filepath, engine='openpyxl', sheet_name=1)
    df = speed.drop([0])
    df.columns = range(df.shape[1])
    df = df.transpose()
    values = df[[isinstance(x, datetime.datetime) for x in df[1]]].copy()
    column_names = df.iloc[0].apply(lambda x: "V_" + str(x))
    column_names[1] = "datetime"
    values.columns = column_names
    values['date'] = [d.date() for d in values['datetime']]
    values['time'] = [d.time() for d in values['datetime']]
    del values["datetime"]
    return values



In [12]:
def get_volume_and_speed_df(filename):
    volume_df = get_volume_df(filename)
    speed_df = get_speed_df(filename)
    merged = volume_df.merge(
        speed_df,
        how="left",
        left_on=["date", "time"],
        right_on=["date", "time"]
    )
    return merged

In [13]:
directory_name = "AFOROS"

xlsx_files = get_xlsx_files(directory_name)

integrated_df = pandas.DataFrame()
df = get_volume_and_speed_df(xlsx_files[0])
df

,date,time,EJE 2 SUR_1 C1,EJE 2 SUR_1 C2,EJE 2 SUR_1 C3,EJE 2 SUR_1 C4,EJE 2 SUR_1 C5,EJE 2 SUR_1 C6,EJE 2 SUR_1 Total,EJE 2 SUR_2 C1,...,V_332,V_333,V_334,V_335,V_336,V_337,V_338,V_339,V_340,V_341
0,2016-01-01,00:00:00,0,799,14,3,5,0,821,332,...,43.91,46.74,35.99,84.28,42.45,41.33,42.93,40.27,48.75,51.7
1,2016-01-01,01:00:00,0,1193,39,12,14,0,1258,464,...,41,47.2,34.93,79.3,42.35,37.31,41.39,38.03,45.38,52.25
2,2016-01-01,02:00:00,0,783,30,2,21,49,885,418,...,39.1,44.91,36.19,78.01,42.37,37.48,40.71,36.35,46.73,50.09
3,2016-01-01,03:00:00,0,759,13,4,34,0,810,332,...,41.72,42.78,34.56,74.33,41.05,38.74,42.16,36.89,46.43,49.99
4,2016-01-01,04:00:00,0,438,6,6,23,0,473,256,...,41.23,44.46,61.08,76.26,39.94,41.8,42.13,38.74,47.27,50.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2016-01-31,19:00:00,0,1055,205,191,85,620,2156,871,...,41.31,48.36,45.88,63.39,45.77,28.59,36.61,35.9,42.66,49.45
740,2016-01-31,20:00:00,0,1115,232,158,96,564,2165,735,...,41.68,47.28,34.56,60.9,45.3,33.09,37.48,36.6,44.43,50.49
741,2016-01-31,21:00:00,0,1021,164,194,87,506,1972,711,...,42.17,48.36,41.17,64.16,48.79,32.18,40.06,37.59,45.14,50.7
742,2016-01-31,22:00:00,0,833,158,121,44,276,1432,535,...,39.83,43.07,47.16,68.94,49.09,33.75,40.8,36.83,45.66,51.4
